## Ejercicio 2

In [3]:
import pandas as pd
# resultado se guarda en muestra2.csv
# Cargar los datos
df = pd.read_csv('train_data.csv')

muestra_por_clase = 10
# Si hay datos etiquetados usar este, si no descomentar el siguiente
df_muestra = df.groupby('clase', group_keys=False).apply(lambda x: x.sample(min(len(x), muestra_por_clase)))
#df_muestra = df.sample(n=10, random_state=70)


# Crear columnas nuevas para el dataset
df_muestra["event"] = ""
df_muestra["aux1"] = ""
df_muestra["category"] = ""
df_muestra["address"] = ""
df_muestra["latitud"] = ""
df_muestra["longitud"] = ""

df_muestra.reset_index(drop=True, inplace=True)
df_muestra.to_csv('muestra2.csv', index=False)

#print(df_muestra.head())

C:\Users\leonardo\AppData\Local\Temp\ipykernel_4148\865782627.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_muestra = df.groupby('clase', group_keys=False).apply(lambda x: x.sample(min(len(x), muestra_por_clase)))


In [4]:
import spacy
from tqdm import tqdm
from transformers import pipeline

# Cargar el modelo de QA
qa_model = pipeline("question-answering", model="timpal0l/mdeberta-v3-base-squad2")
# Cargar el modelo de spaCy
nlp = spacy.load("es_core_news_sm")

def identificar_evento_principal(texto):
    respuesta = qa_model(question="¿Cuál es el evento principal?", context=texto)
    return respuesta['answer']

def extraer_tres_primeras_frases(texto):
    # Tomar una parte de texto para obtener el evento principal sin mucha carga de procesamiento
    frases = texto.split('.')[:5]  # Cambiar a 5
    text = ". ".join(frase.strip() for frase in frases if frase.strip())  
    if text:  
        text += '.'  
    return text

# Obtener evento y guardar
df_muestra["aux1"] = df_muestra["text"].apply(extraer_tres_primeras_frases)  
df_muestra["event"] = df_muestra["aux1"].apply(identificar_evento_principal)  

df_muestra.reset_index(drop=True, inplace=True)
df_muestra.to_csv('muestra2.csv', index=False)


c:\Users\leonardo\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
import spacy
import joblib

nlp = spacy.load("es_core_news_sm")

# Prepocesamiento
def preprocess_text_spacy(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

# Reutilizar los modelos del ejericico 1 para sacar la categoria de la notica
modelo = joblib.load('Model_Ej1_final.pkl') #Entrenamiento con 20k de datos -> 40min
vectorizador = joblib.load('Vect_Ej1_final.pkl')

df_muestra["tokens"] = df_muestra['text'].apply(preprocess_text_spacy) # aplicar preprocesamiento
X_transformed = vectorizador.transform(df_muestra["tokens"])

df_muestra["category"] = modelo.predict(X_transformed) # realizar predicción y guardar

df_muestra.reset_index(drop=True, inplace=True)
df_muestra.to_csv('muestra2.csv', index=False)

In [6]:
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

geolocator = Nominatim(user_agent="Sophia")

def get_addres(text): # Usar el modelo para obtener la ubicación
    question = "En qué lugar fisico ocurre exactamente el evento?"
    context = text
    resp = qa_model(question = question, context = context)
    return resp['answer']

def extraer_loc_con_spacy(texto): # extraer entidades de localización 
    doc = nlp(texto)
    locaciones = [ent.text for ent in doc.ents if ent.label_ == "LOC"]
    return ", ".join(locaciones) if locaciones else None

def obtener_coordenadas(direccion): # Localizar con geopy
    if direccion:
        try:
            location = geolocator.geocode(direccion)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None  
        except GeocoderTimedOut:
            return None, None
    else:
        return None, None  

# Encadenar las 3 funciones usando una columna auxiliar e intentar localizar, si no se retorna none
df_muestra["aux2"] = df_muestra["text"].apply(extraer_loc_con_spacy)
df_muestra["address"] = df_muestra["aux2"].apply(lambda x: get_addres(x) if x else None)
df_muestra["latitud"], df_muestra["longitud"] = zip(*df_muestra["address"].apply(lambda x: obtener_coordenadas(x) if x else (None, None)))

# guardar los datos
df_muestra.reset_index(drop=True, inplace=True)
df_muestra.to_csv('muestra2.csv', index=False)


In [7]:
# Eliminar algunas columnas 
import pandas as pd


# Cargar el archivo CSV
df = pd.read_csv('muestra2.csv')

# Eliminar las columnas no deseadas (ejemplo: 'columna1' y 'columna2')
df = df.drop(['date','media_outlet', 'title','url','text','year','clase','aux1','aux2','tokens'], axis=1)


# Guardar los cambios en un nuevo archivo CSV
df.to_csv('muestra2.csv', index=False)


In [8]:
print("")